In [ ]:
import numpy as np
import cv2
from PIL import Image
from scipy.ndimage.interpolation import rotate
from matplotlib import pyplot as plt
import os

In [ ]:
def load_image(filename):
    return np.asarray(Image.open(filename))

def get_angle(im):
    # Segment image into rectangle corresponding to building
    # and not building
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)    
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    coords = np.column_stack(np.where(thresh > 0))
    
    angle = cv2.minAreaRect(coords)[-1]
    angle = -(90 + angle) if angle < -45 else -angle
    return angle

def rotate_img(im):
    return rotate(im, get_angle(im))

def crop(im):
    x,y,z = np.where(im != 0)
    return im[min(x):max(x)+1,min(y):max(y)+1,min(z):max(z)+1] 

def save_img(im, path):
    result = Image.fromarray(im)
    result.save(path)

In [ ]:
# Utility functions
Path     = os.path.join
DataPath = lambda path: os.path.join(DATA_DIR, path)

def mkdir_p(directory):
    if not os.path.exists(directory):
        os.mkdir(directory)

### TRAIN DATA ALIGN

In [ ]:
DATA_DIR = os.path.join(os.getcwd(), "data")
VERIFIED_DIR   = DataPath("verified")
VERIFIED_ALIGNED_DIR = DataPath("verified_aligned")

mkdir_p(DATA_DIR)
mkdir_p(VERIFIED_DIR)
mkdir_p(VERIFIED_ALIGNED_DIR)

In [ ]:
for class_type in os.listdir(VERIFIED_DIR):
    # Make the aligned images directory
    aligned_path = Path(VERIFIED_ALIGNED_DIR, class_type)
    mkdir_p(aligned_path)
    
    verified_path = Path(VERIFIED_DIR, class_type)
    # Iterate over every image in verified directory
    for im_name in os.listdir(verified_path):
        im_path = Path(verified_path, im_name)
        im = load_image(im_path)
        
        aligned_im = crop(rotate_img(im))
        
        save_img(aligned_im, Path(aligned_path, im_name))

### TEST DATA ALIGN

In [ ]:
TEST_DIR = DataPath("test")
TEST_IMS = Path(TEST_DIR, "test_images")
TEST_ALIGNED_DIR = DataPath("test_aligned")

mkdir_p(TEST_DIR)
mkdir_p(TEST_ALIGNED_DIR)

In [ ]:
for im_name in os.listdir(TEST_IMS):
    im_path = Path(TEST_IMS, im_name)
    im = load_image(im_path)
    
    aligned_im = crop(rotate_img(im))
    save_img(aligned_im, Path(TEST_ALIGNED_DIR, im_name))